# `013` Language Models and Logits

Task: Ask a language model for the most likely next tokens.

This notebook follows up on `012-tokenization`.

## Setup

We'll be using the HuggingFace Transformers library, which provides a (mostly) consistent interface to many different language models. We'll focus on the OpenAI GPT-2 model, famous for OpenAI's assertion that it was "too dangerous" to release in full.

[Documentation](https://huggingface.co/transformers/model_doc/gpt2.html) for the model and tokenizer.

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.2MB 6.7MB/s 
     |████████████████████████████████| 3.3MB 28.1MB/s 
     |████████████████████████████████| 870kB 28.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=0ae5ef7901b56c723d6e2b50464c066cbea5ab83a942265b003894079b3df251
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [ ]:
import torch
from torch import tensor

### Download and load the model

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("distilgpt2", add_prefix_space=True) # smaller version of GPT-2
# Alternative to add_prefix_space is to use `is_split_into_words=True`
# add the EOS token as PAD token to avoid warnings
model = AutoModelForCausalLM.from_pretrained("distilgpt2", pad_token_id=tokenizer.eos_token_id)

In [ ]:
print(f"The model has {model.num_parameters():,d} parameters.")

The model has 81,912,576 parameters.


## Task

Consider the following phrase: "This weekend I plan to".

1. Convert the phrase into token ids.
2. Use the `forward` method of the `model`. Explain the shape of `model_output.logits`.
3. Pull out the logits corresponding to the *last* token in the input phrase. Identify the id of the most likely next token.
4. Find what token the model thinks is the most likely.
5. Use the `topk` method to find the top-20 most likely choices for the next token. 
6. Write a function that is given a phrase and a *k* and returns the top *k* most likely next tokens.

In [ ]:
#phrase = "In a shocking finding, scientists discovered a herd of unicorns living in"
phrase = "This weekend I plan to"

#### Convert phrase to token ids

In [ ]:
input_ids = tokenizer.encode(phrase)
input_ids

[770, 5041, 314, 1410, 284]

#### Use the `forward` method of the `model`

In [ ]:
model_output = model.forward(tensor([input_ids]))
model_output.logits.shape

torch.Size([1, 5, 50257])

The shape of `model_output.logits`:

* 1:: one batch
* 5:: five arrays of next-token-likelihoods for each token in the initial phrase.
* 50257:: the size of the vocabulary; each array contains the likelihoods of being the next token for each token in the vocabulary.


In [ ]:
# since we only have a single sequence (batch size of 1), let's collapse the batch dimension.
logits = model_output.logits[0]

#### Pull out the logits corresponding to the last token in the input phrase

In [ ]:
last_token_logits = logits[-1]
last_token_logits

tensor([-77.8725, -79.7684, -82.1183,  ..., -88.5235, -86.5615, -79.6716],
       grad_fn=<SelectBackward>)

#### Identify the id of the most likely next token

In [ ]:
max_logit = last_token_logits.max()
most_likely_id = (last_token_logits == max_logit).nonzero(as_tuple=True)[0]
most_likely_id

tensor([467])

#### Find what token the model thinks is the most likely

In [ ]:
tokenizer.decode(most_likely_id)

' go'

#### Find the top-20 most likely choices for the next token

In [ ]:
top_tokens = last_token_logits.topk(k=20)
tokenizer.decode(top_tokens[1])

' go take spend make do be attend visit run have write play get start head travel try bring return share'

#### Write a function that is given a phrase and a k and returns the top k most likely next tokens

In [ ]:
def get_top_tokens(phrase, k):
  input_ids = tokenizer.encode(phrase)
  model_output = model.forward(tensor([input_ids]))
  top_tokens = model_output.logits[0][-1].topk(k=k)
  return tokenizer.decode(top_tokens[1])

get_top_tokens("I wish I were", 10)

' a the more in there able here an on just'

## Analysis

What would be required to generate more than one token? What decisions would you have to make?

To generate more than one token, we would have to add a token onto the phrase and feed that phrase back into the model to generate more possible tokens. We would need to decide which token to append to the original phrase. This could be the token the model believes is the most likely choice, or it could be chosen randomly across the top *k* tokens. Either way, when we input the new phrase back into the model, the model will generate new tokens based on the last token we added.